In [ ]:
import sparrow as sp

In [ ]:
import os
import uuid
import tempfile
from sparrow.datasets.registry import get_registry

registry = get_registry()  # on the cluster fetch the results in other path

_ = registry.fetch("transcriptomics/vizgen/mouse/Liver1Slice1/images/mosaic_DAPI_z3.tif")
_ = registry.fetch("transcriptomics/vizgen/mouse/Liver1Slice1/images/mosaic_PolyT_z3.tif")
_ = registry.fetch("transcriptomics/vizgen/mouse/Liver1Slice1/images/micron_to_mosaic_pixel_transform.csv")
path_transcripts = registry.fetch("transcriptomics/vizgen/mouse/Liver1Slice1/detected_transcripts.csv")

input_path = os.path.dirname(path_transcripts)
OUTPUT_DIR =  tempfile.gettempdir()

# takes around 5m on mac M2
sdata = sp.io.merscope(
    path=input_path,
    to_coordinate_system="global",
    z_layers=[
        3,
    ],
    backend=None,
    transcripts=True,
    mosaic_images=True,
    do_3D=False,
    z_projection=False,
    image_models_kwargs={"scale_factors": None},
    output=os.path.join( OUTPUT_DIR, f"sdata_merscope_{uuid.uuid4()}.zarr"),
    filter_gene_names=[ "blank" ]
)

In [ ]:
from spatialdata import read_zarr

sdata=read_zarr(sdata.path)

In [ ]:
sdata

In [ ]:
sdata=sp.im.min_max_filtering(
    sdata,
    img_layer="mouse_Liver1Slice1_z3_global",
    output_layer="min_max_filtered",
    size_min_max_filter=[ 85, 135 ],
    crd = [  20000, 25000, 20000, 25000 ],
    overwrite=True,
      )
sdata=sp.im.enhance_contrast(
    sdata,
    img_layer="min_max_filtered",
    output_layer="clahe",
    contrast_clip=[13.5, 18.5 ],
    crd = [  20000, 25000, 20000, 25000 ],
    overwrite=True,
      )

In [ ]:
sp.pl.plot_shapes( sdata, img_layer=["mouse_Liver1Slice1_z3_global", "clahe"], crd = [ 20000,25000,20000,25000 ], figsize=(10,10) )  # do some segmentation here. Test different settings on hpc

In [ ]:
#from dask.distributed import Client, LocalCluster

#cluster = LocalCluster(
#    n_workers=8,
#    threads_per_worker=1,
#    processes=True,
#    memory_limit="32GB",
#)

#client = Client(cluster)

#print(client.dashboard_link)

In [ ]:
import torch
from cellpose import models

from sparrow.image import cellpose_callable

from sparrow.image._image import _get_spatial_element

if torch.backends.mps.is_available():
    device = "mps"
    gpu = True
elif torch.cuda.is_available():
    device = "cuda"
    gpu = True
else:
    device = "cpu"
    gpu = False

model=models.CellposeModel( gpu=gpu, pretrained_model='cyto3', device = torch.device(device ) )

#model = client.scatter(model) # pass a loaded model to _cellpose, but we scatter the model to avoid large task graph

se = _get_spatial_element( sdata, layer = "clahe" )

sdata = sp.im.segment(
    sdata,
    img_layer="clahe",
    chunks=2048,
    depth=200,
    model=cellpose_callable,
    # parameters that will be passed to the callable _cellpose
    pretrained_model = model,
    diameter=100,
    flow_threshold=0.85,
    cellprob_threshold=-4,
    channels = [ se.c.data.tolist().index("PolyT" )+1  ,se.c.data.tolist().index("DAPI" )+1 ],
    output_labels_layer="segmentation_mask",
    output_shapes_layer="segmentation_mask_boundaries",
    crd= [20000, 25000, 20000, 25000],  # region to segment [x_min, xmax, y_min, y_max],
    overwrite=True,
)


In [ ]:
sp.pl.plot_shapes( sdata, shapes_layer="segmentation_mask_boundaries" , img_layer=["clahe"], crd = [ 20000,25000,20000,25000 ], figsize=( 10,10 ) )

In [ ]:
sdata = sp.tb.allocate(
    sdata=sdata,
    labels_layer="segmentation_mask",
    points_layer="transcripts_global",
    output_layer="table_transcriptomics",
    update_shapes_layers=False,
    overwrite=True,
)

In [ ]:
sp.pl.sanity_plot_transcripts_matrix(
    sdata,
    img_layer="clahe",
    shapes_layer = "segmentation_mask_boundaries",
    points_layer= "transcripts_global",
    plot_cell_number=True,
    gene="Vwf",
    crd = [ 20500, 20500+500, 21000, 21500 ],
    figsize=(5,5),
)

In [ ]:
sp.pl.plot_shapes(
    sdata,
    img_layer="clahe",
    shapes_layer="segmentation_mask_boundaries",
    figsize=( 10,10 ),
    crd = [  20000, 22000, 20000, 22000  ],
    table_layer="table_transcriptomics",
    column = "Vwf",
      )

In [ ]:
df = sp.pl.analyse_genes_left_out(
    sdata,
    labels_layer="segmentation_mask",
    table_layer="table_transcriptomics",
    points_layer="transcripts_global",
)

In [ ]:
# Perform preprocessing.
sdata = sp.tb.preprocess_transcriptomics(
    sdata,
    labels_layer="segmentation_mask",
    table_layer="table_transcriptomics",
    output_layer="table_transcriptomics_preprocessed",  # write results to a new slot, we could also write to the same slot (when passing overwrite==True).
    min_counts=10,
    min_cells=5,
    size_norm=True,
    n_comps=50,
    overwrite=True,
    update_shapes_layers=False,
)

In [ ]:
sdata[ "table_transcriptomics_preprocessed" ]

In [ ]:
sp.pl.preprocess_transcriptomics(
    sdata,
    table_layer="table_transcriptomics_preprocessed",
)

In [ ]:
sdata = sp.tb.filter_on_size(
    sdata,
    labels_layer="segmentation_mask",
    table_layer="table_transcriptomics_preprocessed",
    output_layer="table_transcriptomics_filter",
    min_size=500,
    max_size=100000,
    update_shapes_layers=False,
    overwrite=True,
)

In [ ]:
import scanpy as sc

sdata = sp.tb.leiden(
    sdata,
    labels_layer="segmentation_mask",
    table_layer="table_transcriptomics_filter",
    output_layer="table_transcriptomics_clustered",
    calculate_umap=True,
    calculate_neighbors=True,
    n_pcs=17,
    n_neighbors=35,
    resolution=1.0,
    rank_genes=True,
    key_added="leiden",
    overwrite=True,
)

sc.pl.umap(sdata.tables["table_transcriptomics_clustered"], color=["leiden"], show=True)
sc.pl.rank_genes_groups(sdata.tables["table_transcriptomics_clustered"], n_genes=8, sharey=False, show=True)

In [ ]:
sp.pl.plot_shapes(
    sdata,
    img_layer="clahe",
    table_layer="table_transcriptomics_clustered",
    column="leiden",
    shapes_layer="segmentation_mask_boundaries",
    alpha=1,
    linewidth=0,
    channel="DAPI",
    crd = [ 20000,25000,20000,25000 ],
)